In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from splinter import Browser
import re

In [3]:
# Initialize PyMongo to work with MongoDBs(local host for Mongodb always 27017)
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.NASA_db
collection = db.items

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


# URL of page to be scraped -NASA Mars News
url1 = 'https://mars.nasa.gov/news/'
browser.visit(url1)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(browser.html, 'lxml')

# Get the message and message inside headlines

In [6]:
#to get the headlines
title = soup.find('div',{'class' : 'content_title'})
title. text

'Curiosity on the Move Again'

In [7]:
# to get the message inside the headlines
para= soup.find('div', {'class':'article_teaser_body'})
para.text


"NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles."

# URL of page to be scraped-JPL Mars Space Images - Featured Image

In [8]:
# URL of page to be scraped-JPL Mars Space Images - Featured Image
url2 ='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(browser.html, 'lxml')

In [9]:
carousel0 = soup.find('div',{'class' : 'carousel_container'})
carousel0

<div class="carousel_container">
<div class="carousel_items">
<article alt="Stars Brewing in Cygnus X" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA15253-1920x1200.jpg');">
<div class="default floating_text_area ms-layer">
<h2 class="category_title">
</h2>
<h2 class="brand_title">
				  FEATURED IMAGE
				</h2>
<h1 class="media_feature_title">
				  Stars Brewing in Cygnus X				</h1>
<div class="description">
</div>
<footer>
<a class="button fancybox" data-description="A bubbling cauldron of star birth is highlighted in this image from NASA's Spitzer Space Telescope. Massive stars have blown bubbles, or cavities, in the dust and gas -- a violent process that triggers both the death and birth of stars." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA15253_ip.jpg" data-link="/spaceimages/details.php?id=PIA15253" data-title="Stars Brewing in Cygnus X" id="full_image">
					FULL IMAGE
				  </a>
</footer>
</div>
<d

In [10]:
carousel1 = carousel0.find('article', {'class': 'carousel_item'})
carousel1


<article alt="Stars Brewing in Cygnus X" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA15253-1920x1200.jpg');">
<div class="default floating_text_area ms-layer">
<h2 class="category_title">
</h2>
<h2 class="brand_title">
				  FEATURED IMAGE
				</h2>
<h1 class="media_feature_title">
				  Stars Brewing in Cygnus X				</h1>
<div class="description">
</div>
<footer>
<a class="button fancybox" data-description="A bubbling cauldron of star birth is highlighted in this image from NASA's Spitzer Space Telescope. Massive stars have blown bubbles, or cavities, in the dust and gas -- a violent process that triggers both the death and birth of stars." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA15253_ip.jpg" data-link="/spaceimages/details.php?id=PIA15253" data-title="Stars Brewing in Cygnus X" id="full_image">
					FULL IMAGE
				  </a>
</footer>
</div>
<div class="gradient_container_top"></div>
<div class="gradient_

In [11]:
carousel3 = carousel1['style'].strip()
print(carousel3)
carousel4 = re.sub('[(){}]<>]','',carousel3)
print(carousel4)
result = re.search('background-image: url(.*);',carousel4)
result
carousel5 = result.group(1).strip('\'')
output = {}
output['featured_img_url']= 'https://www.jpl.nasa.gov'+ carousel5
print (output)

background-image: url('/spaceimages/images/wallpaper/PIA15253-1920x1200.jpg');
background-image: url('/spaceimages/images/wallpaper/PIA15253-1920x1200.jpg');
{'featured_img_url': "https://www.jpl.nasa.gov('/spaceimages/images/wallpaper/PIA15253-1920x1200.jpg')"}


#### Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page.


In [16]:
url3 = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)
soup = bs(browser.html, 'lxml')
MarsWeather= soup.find('p',class_="tweet-text").text
Mars_Weather = {'Mars Weather' : MarsWeather}
print(Mars_Weather)

{'Mars Weather': 'Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36'}


# Mars facts

In [19]:
url4 = "https://space-facts.com/mars/"
table = pd.read_html(url4)
print (table)


[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers]


In [20]:
df = table[0]
df.columns =["description","value"]
df.set_index("description")
table = df.to_html(classes = ['table-table-striped'])
mars_table = {'table': table}
print(mars_table)

{'table': '<table border="1" class="dataframe table-table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:<

# Mars Hemispheres

In [28]:
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url5)
soup = bs(browser.html, 'lxml')

In [34]:
results = soup.find_all(class_ = 'item')
marsHemList = []
marsHem ={}
for x in results :
    img = x.find('img')
    img_url = img ['src']
    title = x.find('h3').text
    marsHem.update({title:('https://astrogeology.usgs.gov' + img_url)})
    marsHem ={"title": title, "img_url":('https://astrogeology.usgs.gov' + img_url)}
    marsHemList.append(marsHem)
print(marsHemList)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png', 'Schiaparelli Hemisphere Enhanced': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png', 'Syrtis Major Hemisphere Enhanced': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png', 'Valles Marineris Hemisphere Enhanced': 'https://astrogeology.usgs.gov/cache/images/04085d99ec3713883a9a57f42be9c725_valles_marineris_enhanced.tif_thumb.png'}, {'title': 'Valles Marineris